In [1]:
%pip install --upgrade caer canaro

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.5/809.5 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 2.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.1/418.1 kB 3.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os 
import caer 
import canaro 
import numpy as np 
import cv2 as cv 
import gc 

In [ ]:
IMG_SIZE = (80, 80)
channels = 1 
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
# Creating a character dictionary, sorting it in descending order 
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
    
# Sort in descending order 
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict 

In [ ]:
# Getting the first 10 categories with thye most number of images 
characters = []
count = 0 
for i in char_dict:
    characters.append(i[0])
    count += 1 
    if count >= 10: 
        break 
characters 

In [ ]:
# Create the training data 
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
# Number of training samples 
len(train)

In [ ]:
# Visualizing the data (OpenCV doesn't display wel in Jupyter notebooks)
import matplotlib.pyplot as plt 
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [ ]:
# Separating the array and corresponding labels 
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical 

# Normalize the featureSet ==> (0,1)
featureSet = caer.normalize(featureSet)
# Converting numerical labels to binary class vectors 
labels = to_categorical(labels, len(characters))

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [ ]:
del train 
del featureSet 
del labels 
gc.collect()

In [ ]:
# Useful variables when training 
BATCH_SIZE = 32 
EPOCHS = 10 

In [ ]:
# Image data generator (introduces randomness in network ==> better accuracy)
import canaro.generators


datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)